In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('1128_output.csv')

In [3]:
# 找出TAICCA_ID中沒有「/」且min_publish_date為空的資料
mask = (~df['TAICCA_ID'].str.contains('/', na=False)) & (df['min_publish_date'].isna())

# 對這些資料，從各平台的發布日期中找出有值的日期來填補
for idx in df[mask].index:
    # 收集各平台的發布日期
    dates = []
    for col in ['bookscom_publish_date', 'eslite_publish_date', 'kingstone_publish_date', 'sanmin_publish_date']:
        if pd.notna(df.loc[idx, col]) and str(df.loc[idx, col]).strip():
            dates.append(df.loc[idx, col])
    
    # 如果有找到日期，就填入min_publish_date和max_publish_date
    if dates:
        # 直接使用第一個找到的日期，不進行比較
        df.loc[idx, 'min_publish_date'] = dates[0]
        df.loc[idx, 'max_publish_date'] = dates[0]

print(f"已處理 {mask.sum()} 筆資料的發布日期")


已處理 5966 筆資料的發布日期


In [7]:
# 統一min_publish_date和max_publish_date的格式為YYYY-MM-DD
from datetime import datetime
import re

def standardize_date_format(date_str):
    """將各種日期格式統一為YYYY-MM-DD格式"""
    if pd.isna(date_str) or not str(date_str).strip():
        return date_str
    
    date_str = str(date_str).strip()
    
    # 如果已經是YYYY-MM-DD格式，直接返回
    if re.match(r'^\d{4}-\d{2}-\d{2}$', date_str):
        return date_str
    
    # 處理各種可能的日期格式
    try:
        # 嘗試解析M/D/YY格式 (如 7/8/25)
        if re.match(r'^\d{1,2}/\d{1,2}/\d{2}$', date_str):
            month, day, year = date_str.split('/')
            # 假設YY格式的年份都是20XX年
            year = f"20{year}"
            return f"{year}-{month.zfill(2)}-{day.zfill(2)}"
        
        # 嘗試解析YYYY-M-D格式
        if re.match(r'^\d{4}-\d{1,2}-\d{1,2}$', date_str):
            parts = date_str.split('-')
            return f"{parts[0]}-{parts[1].zfill(2)}-{parts[2].zfill(2)}"
        
        # 嘗試其他常見格式
        for fmt in ['%Y-%m-%d', '%Y/%m/%d', '%m/%d/%Y', '%d/%m/%Y']:
            try:
                parsed_date = datetime.strptime(date_str, fmt)
                return parsed_date.strftime('%Y-%m-%d')
            except ValueError:
                continue
                
    except Exception as e:
        print(f"無法解析日期格式: {date_str}")
        return date_str
    
    return date_str

# 處理min_publish_date和max_publish_date欄位
for col in ['min_publish_date', 'max_publish_date', 'bookscom_publish_date', 'eslite_publish_date', 'kingstone_publish_date', 'sanmin_publish_date']:
    if col in df.columns:
        df[col] = df[col].apply(standardize_date_format)

print("已統一日期格式為YYYY-MM-DD")

已統一日期格式為YYYY-MM-DD


In [9]:
# 以title進行A-Z排序
# 首先檢查是否有title相關的欄位
title_columns = [col for col in df.columns if 'title' in col.lower()]
print(f"找到的title相關欄位: {title_columns}")

# 使用主要的title欄位進行排序，如果沒有則使用第一個找到的title欄位
if 'title' in df.columns:
    sort_column = 'title'
elif 'bookscom_title' in df.columns:
    sort_column = 'bookscom_title'
elif title_columns:
    sort_column = title_columns[0]
else:
    print("警告: 找不到title欄位，將使用第一個欄位進行排序")
    sort_column = df.columns[0]

print(f"使用 '{sort_column}' 欄位進行A-Z排序")

# 進行排序，將空值放在最後
df = df.sort_values(by=sort_column, ascending=True, na_position='last')

# 重置索引
df = df.reset_index(drop=True)

print(f"排序完成，共 {len(df)} 筆資料")


找到的title相關欄位: ['title', 'bookscom_title', 'eslite_title', 'kingstone_title', 'sanmin_title', 'processed_title', 'bookscom_processed_title', 'eslite_processed_title', 'kingstone_processed_title', 'sanmin_processed_title', 'original_title', 'bookscom_original_title', 'eslite_original_title', 'kingstone_original_title', 'sanmin_original_title']
使用 'title' 欄位進行A-Z排序
排序完成，共 10260 筆資料


In [10]:
df.to_csv('1201_output.csv', index=False, encoding='utf-8')